In [53]:
import torch
import clip
from PIL import Image

In [55]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
model, preprocess = clip.load("RN50", device=device)
# print("")
# model, preprocess = clip.load("RN101", device=device)
# print("")
# model, preprocess = clip.load("RN50x4", device=device)
# print("")
# model, preprocess = clip.load("RN50x16", device=device)
# print("")
# model, preprocess = clip.load("RN50x64", device=device)

# model, preprocess = clip.load("ViT-B/32", device=device)

In [19]:
image1 = preprocess(Image.open("CLIP.png"))
image2 = preprocess(Image.open("city.png"))
image = torch.stack([image1, image2])
classes = ["diagram", "dog", "cat", "animal", "book", "city"]
text = torch.cat([clip.tokenize(f"a photo of a {c}") for c in classes]).to(device)

In [28]:
image_features = model.encode_image(image)
text_features = model.encode_text(text)
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = image_features @ text_features.T
image_features.shape, text_features.shape, similarity.sigmoid()

(torch.Size([2, 1024]),
 torch.Size([6, 1024]),
 tensor([[0.5592, 0.5441, 0.5424, 0.5448, 0.5420, 0.5347],
         [0.5394, 0.5380, 0.5371, 0.5393, 0.5398, 0.5548]],
        grad_fn=<SigmoidBackward0>))

In [43]:
probs_per_image = logits_per_image.softmax(dim=-1)
probs_per_text = logits_per_text.softmax(dim=-1)
target = torch.tensor([[1,0,0,0,0,0],[0,0,0,0,0,1]]).type(torch.float32)
loss = torch.nn.CrossEntropyLoss()
loss(probs_per_image, target), loss(probs_per_text, target.T)

(tensor(1.0495, grad_fn=<DivBackward1>),
 tensor(0.1045, grad_fn=<DivBackward1>))

In [33]:
# Calculate features
with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

# Pick the top 5 most similar labels for the image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
values, indices = similarity[0].topk(5)

# Print the result
print("\nTop predictions:\n")
for value, index in zip(values, indices):
    print(f"{classes[index]:>16s}: {100 * value.item():.2f}%")


Top predictions:

         diagram: 98.20%
           paper: 1.09%
          animal: 0.29%
             dog: 0.22%
             cat: 0.11%


In [34]:
image_features.shape, text_features.shape

(torch.Size([2, 1024]), torch.Size([6, 1024]))

In [35]:
logits_per_image = (100.0 * image_features @ text_features.T).softmax(dim=-1)
target = torch.tensor([[1,0,0,0,0,0]]).type(torch.float32)
torch.nn.CrossEntropyLoss()(similarity, target)

ValueError: Expected input batch_size (2) to match target batch_size (1).

In [26]:
similarity

tensor([[9.8203e-01, 2.2167e-03, 1.0847e-03, 2.8931e-03, 9.2102e-04, 1.0857e-02]])

In [29]:
similarity.shape, target.shape

(torch.Size([1, 6]), torch.Size([1, 6]))

In [40]:
target = torch.randint(0,2,(3,5), dtype=torch.float32)
target

tensor([[1., 0., 1., 1., 1.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 1.]])

In [42]:
output = torch.rand((3,5))
output

tensor([[0.6745, 0.5993, 0.1119, 0.3808, 0.3173],
        [0.5056, 0.6278, 0.9587, 0.3396, 0.8850],
        [0.0677, 0.2938, 0.0428, 0.8069, 0.3061]])

In [45]:
import torch.nn as nn
multi_criterion = nn.MultiLabelSoftMarginLoss(reduction='none')
multi_criterion(output, target)

tensor([0.6311, 0.8931, 0.6428])

In [47]:
bce_criterion = nn.BCEWithLogitsLoss(reduction='none')
bce_criterion(output, target)

tensor([[0.4117, 1.0370, 0.6388, 0.5208, 0.5470],
        [0.9776, 1.0556, 0.3246, 0.8773, 1.2305],
        [0.7276, 0.8508, 0.7148, 0.3690, 0.5517]])

In [46]:
bce_criterion = nn.BCEWithLogitsLoss(reduction='none')
bce_criterion(output, target).mean(axis=-1)

tensor([0.6311, 0.8931, 0.6428])